In [ ]:
paths = [
"outputs/2023-10-18/11-44-08-805221",
# "outputs/2023-11-01/16-24-19-171848",
"outputs/2023-10-18/11-44-08-874258",
"outputs/2023-10-18/11-44-08-874396",
"outputs/2023-10-18/11-44-08-882528",
"outputs/2023-10-18/11-44-08-884445",
"outputs/2023-10-18/11-44-08-886024",
"outputs/2023-10-18/11-44-08-906468",
"outputs/2023-10-18/11-44-08-911000",
"outputs/2023-10-18/11-44-08-932201",
"outputs/2023-10-18/11-44-08-953521",
# "outputs/2023-11-12/19-20-15-010722",
"outputs/2023-11-13/12-05-39-054691",
"outputs/2023-11-14/10-27-23-673379"
]

In [ ]:
paths2 = [
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/11-44-53-320622",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/11-44-53-041944",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/11-44-53-295893",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/11-44-53-403698",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/11-44-52-854931",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/12-21-36-646480",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/12-21-36-588119",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/12-27-29-253904",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/12-21-36-614857",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/12-00-28-036885",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/11-44-53-222033",
"/raid/lingo/akyurek/git/iclmodels/outputs/2023-11-15/11-44-53-201063",
]

In [ ]:
import os
import yaml
from yaml import Loader
import glob

In [ ]:
import pickle
import matplotlib.pyplot as plt
from probe import get_results
import os

In [ ]:
import glob
exp_folders = {}
for path in paths2:
    yaml_file = os.path.join(path, ".hydra", "overrides.yaml")
    # parse file
    with open(yaml_file, 'r') as stream:
        try:
            data = yaml.safe_load(stream)
            print(data)
            experiment = data[-1]
            experiment = experiment.split("=")[1]
            experiment = experiment.replace("dfa/", "")
            if experiment == "lstm":
                layer = data[4]
            else:
                layer = data[5]
            layer = layer.split("=")[1]
            exp_folders[f"{experiment}/{layer}"] = path
            print("export PYTHONHASHSEED=0; export CUDA_VISIBLE_DEVICES=0; python train.py model.return_attention=True" + " ".join(data))
            # ckpt = glob.glob(os.path.join(path, "finalexps2", "*/checkpoints/*.ckpt"))[0]
            # # escape equal sign
            # ckpt = ckpt.replace("=", "\\=")
            # overrides = " ".join(data)
            # print(f"export PYTHONHASHSEED=0; export CUDA_VISIBLE_DEVICES=2; python train.py train.ckpt=\"{ckpt}\" train.test=true {overrides} > newexps/{experiment}_{layer}.log 2> newexps/{experiment}_{layer}.err &")
        except yaml.YAMLError as exc:
            print(exc)

In [ ]:
exp_folders

In [ ]:
# video_files = """outputs/2023-10-18/11-44-08-805221/res_lw_True_videos TF8
# outputs/2023-10-18/11-44-08-874258/res_lw_True_videos TF2
# outputs/2023-10-18/11-44-08-874396/res_lw_True_videos TF4
# outputs/2023-10-18/11-44-08-884445/res_lw_True_videos LTF4
# outputs/2023-10-18/11-44-08-886024/res_lw_True_videos RT4
# outputs/2023-10-18/11-44-08-906468/res_lw_True_videos RWKV
# outputs/2023-10-18/11-44-08-911000/res_lw_True_videos H3
# outputs/2023-10-18/11-44-08-932201/res_lw_True_videos HYENA
# outputs/2023-11-13/12-05-39-054691/res_lw_True_videos TF12
# outputs/2023-11-14/10-27-23-673379/res_lw_True_videos LTF8"""

In [ ]:
# # copy videos under the folders to a named folder
# import os
# import shutil
# import glob
# import re

# os.makedirs("video_artifacts", exist_ok=True)
# for line in video_files.split("\n"):
#     folder, name = line.split(" ")
#     video_folder = os.path.join(folder, "res_lw_True_videos")
#     os.makedirs("video_artifacts/" + name, exist_ok=True)
#     # copy video folder to new named folder using cp
#     os.system(f"cp -r {video_folder} video_artifacts/{name}/")


## In-context learning curve of each model

In [ ]:
def in_context_learning_curve(folder, title="null"):
    data = get_results(folder + "/generations/200_test.txt")
    accs = []
    for t in range(300):
        total = 0.0
        correct = 0.0
        for b in range(len(data)):
            labels = data[b]["char_labels"]
            if len(labels) > t:
                correct += labels[t]
                total += 1
        accs.append(correct / total)
    # plot accs
    # make high res
    if title in ("transformer/4", "transformer/2", "transformer/1"):
        return None
    title = title.split("/")[0]
    # make first letter uppercase
    title = title[0].upper() + title[1:]
    title = title.replace("_", " ")


    plt.plot(accs, label=title)
    # set x label
    plt.legend()


In [ ]:
data1 = get_results(exp_folders["transformer/8"] + "/generations/200_test.txt")

In [ ]:
data1[0].keys()

In [ ]:
print(data1[0]["vocab"])

In [ ]:
print(data1[0]["input"])

In [ ]:
def write_dfa_data(data):
    for index, datum in enumerate(data):
        input = datum["input"]
        examples = input.split("|")
        vocab = sorted(list(set(list("".join(examples)))))
        inv_vocab = {v: i for i, v in enumerate(vocab)}
        with open(f"FlexFringe/data/icl_dfa_data/{index}.dat", "w") as handle:
            print(f"{len(examples)} {len(vocab)}", file=handle)
            for example in examples:
                encoded = [inv_vocab[c] for c in example]
                encoded_str = ' '.join([str(c) for c in encoded])
                print(f"1 {len(encoded)} {encoded_str}", file=handle)
        # print vocab information to another file with same index
        # with open(f"FlexFringe/data/icl_dfa_data/{index}_vocab.txt", "w") as handle:
        #     vocab_str = ' '.join(vocab)
        #     print(f"{vocab_str}", file=handle)

In [ ]:
write_dfa_data(data1)

In [ ]:
data2 = get_results(exp_folders["h3/2"] + "/generations/200_test.txt")

In [ ]:
for d1, d2 in zip(data1, data2):
    assert d1["input"] == d2["input"]

In [ ]:
# configure matplotlib to make high-quality conference style
rc = {
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 10,
    "axes.labelsize": 12,
    "axes.titlesize": 12,
    "legend.fontsize": 10,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "figure.figsize": [3.39, 2.10],
    "figure.dpi": 300,
    "text.usetex": True,
    "text.latex.preamble": r"\usepackage{amsmath}\usepackage{amssymb}",
}
# set rc
plt.rcParams.update(rc)


In [ ]:
# plot learning curves
plt.figure(figsize=(8, 6))
# add xlabel
plt.xlabel("number of characters in context")
# set y label
plt.ylabel("next character validity")
# plt.title("In-context Accuracy")
for exp  in exp_folders.keys():
    try:
        in_context_learning_curve(exp_folders[exp], title=exp)
    except FileNotFoundError:
        print(f"skipping {exp}")

## n-gram vs model; model vs dfa; dfa vs n-gram

In [ ]:
import torch
from analyze import get_dfa_probs as calculate_dfa_probs
from ngram import (
    predict_with_n_gram_back_off,
    prob_distance,
    prob_distance_dfa,
    prob_distance_dfa_ngram,
)

from batched_baum_welch import predict_with_baumwelch

import matplotlib.pyplot as plt
import numpy as np


class Vocab:
    def __init__(self, vocab: list):
        self.vocab = vocab
        # inverse vocab
        self.inv_vocab = {v: k for k, v in enumerate(vocab)}

    def get_vocab(self, id):
        return self.vocab[id]

    def get_id(self, char):
        return self.inv_vocab[char]

    def __len__(self):
        return len(self.vocab)

def get_ngram_probs(results, ngram=3, uniform=False, backoff=False, addone=False):
    vocab = Vocab(results[0]["vocab"])
    n_gram_probs = []
    for b in range(len(results)):
        input = results[b]["input"]
        target = [vocab.get_id(t) for t in results[b]["target"]]
        probs = predict_with_n_gram_back_off(input, N=ngram, global_vocab=vocab, uniform=uniform, backoff=backoff, addone=addone)
        n_gram_probs.append(probs)
    return n_gram_probs

def get_baumwelch_probs(results):
    vocab = Vocab(results[0]["vocab"])
    baumwelch_probs = []
    for b in range(len(results)):
        input = results[b]["input"]
        probs = predict_with_baumwelch(input, vocab, max_states=12)
        baumwelch_probs.append(probs)
    return baumwelch_probs

def get_dfa_probs(results):
    vocab = Vocab(results[0]["vocab"])
    dfa_probs = []
    for b in range(len(results)):
        input = results[b]["input"]
        target = [vocab.get_id(t) for t in results[b]["target"]]
        probs = calculate_dfa_probs(input, results[b]["dfa"], vocab=vocab)
        dfa_probs.append(probs)
    return dfa_probs

def get_model_probs(results):
    model_probs = []
    for b in range(len(results)):
        probs = torch.softmax(torch.tensor(results[b]["probs"]), dim=-1).detach().cpu().numpy()
        model_probs.append(probs)
    return model_probs

In [ ]:
import numpy as np
def get_greedy_dfa_accuracy(probs, dfa_probs):
    total = 0.0
    correct = 0.0
    for p1, pdfa in zip(probs, dfa_probs):
        indices = p1.argmax(axis=-1)[:len(pdfa)]
        correct += (pdfa[np.arange(len(pdfa)), indices] > 0).sum()
        total += len(pdfa)
    return correct / total

In [ ]:
EPS=1e-7
def get_cross_entropy(probs, dfa_probs):
    total = 0.0
    cross_entropy = 0.0
    for p1, pdfa in zip(probs, dfa_probs):
        # calculate the soft cross-entropy between p1 and pdfa
        log_p1 = np.log(p1[:len(pdfa)] + EPS)
        log_pdfa = np.log(pdfa + EPS)
        cross_entropy += -((log_p1 - log_pdfa) * pdfa).sum()
        total += len(pdfa)
    return cross_entropy / total

In [ ]:
def get_l1_loss(probs1, probs2, probsdfa, offset=0, max_len=None):
    total = 0.0
    correct = 0.0
    for p1, p2, pdfa in zip(probs1, probs2, probsdfa):
        if max_len is not None:
            pdfa = pdfa[offset:max_len]
        total += len(pdfa)
        correct += np.abs(p1[offset:offset + len(pdfa)] - p2[offset:offset + len(pdfa)]).sum()
    return correct / total

In [ ]:
data1 = get_results(exp_folders["transformer/8"] + "/generations/200_test.txt")

In [ ]:
probs = {}

In [ ]:
probs["dfa"] = get_dfa_probs(data1)

In [ ]:
probs["bw"]  = get_baumwelch_probs(data1)

In [ ]:
get_greedy_dfa_accuracy(probs["bw"] , probs["dfa"])

In [ ]:
get_cross_entropy(probs["bw"] , probs["dfa"])

In [ ]:
probs["3gram"] = get_ngram_probs(data1, 3, uniform=False, addone=False, backoff=True)
probs["2gram"] = get_ngram_probs(data1, 2, uniform=False, addone=False, backoff=True)
# probs["3gram2gram1"] = [(0.7 * probs3 + 0.3 * probs2) for probs3, probs2 in zip(probs["3gram"], probs["2gram"])]
# probs["3gram2gram2"] = [(0.5 * probs3 + 0.5 * probs2) for probs3, probs2 in zip(probs["3gram"], probs["2gram"])]
# probs["3gram2gram3"] = [(0.3 * probs3 + 0.7 * probs2) for probs3, probs2 in zip(probs["3gram"], probs["2gram"])]
probs["3gramU"] = get_ngram_probs(data1, 3, uniform=True, addone=False, backoff=True)
probs["2gramU"] = get_ngram_probs(data1, 2, uniform=True, addone=False, backoff=True)


In [ ]:
# probs.pop("3gram2gram1")
# probs.pop("3gram2gram2")
# probs.pop("3gram2gram3")

In [ ]:
get_greedy_dfa_accuracy(probs["3gram"] , probs["dfa"])

In [ ]:
# probs["3gram"][0][200]

In [ ]:
for exp_name, folder in exp_folders.items():
    print(exp_name)
    try:
        data = get_results(folder + "/generations/200_test.txt")[:20]
        probs[exp_name] = get_model_probs(data)
    except:
        print(f"skipping {exp_name}")

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
def logitsmap(current_probs, chars, vocab, model="transformer/8", id=0, offset=200):
    # l1_losses is a dict of (key1, key2) -> l1_loss
    # we want to create a df where key1s are the index
    # and key2s are the columns

    plt.figure(figsize=(30, 20))

    key1s = vocab
    # key2s = list(map(str, enumerate(chars[200:300])))
    chars = chars[offset:offset+100]
    indices = list(range(offset, offset+len(chars)))
    key2s = list(map(str, zip(indices, chars)))

    plot_index = 0
    ax1 = None
    for name, prob in current_probs.items():
        df = pd.DataFrame(index=key1s, columns=key2s)
        for index, key2 in enumerate(key2s):
            df.loc[:, key2] = prob[offset + index]
        # replace nans with 0
        df = df.fillna(0)
        # heatmap
        ax = plt.subplot(len(current_probs), 1, plot_index + 1)
        if ax1 is None:
            ax1 = ax
            ax.xaxis.tick_top()
        else:
            # remove xticks
            ax.xaxis.set_visible(False)

        sns.heatmap(df, annot=False, ax=ax, vmin=0, vmax=1)

        ax.sharex(ax1)


        # add title
        plt.title(f"{name}")
        plot_index += 1

    os.makedirs(f"attention_plots/{model}", exist_ok=True)
    path = os.path.join("attention_plots", f"{model}", f"{id}_{offset}_{offset + len(chars)}_probs.png")
    plt.savefig(path, dpi=250)
    return path

In [ ]:
for exp_name, folder in exp_folders.items():
    print(exp_name)
    try:
        for index in range(2):
            modelprobs = probs[exp_name][index]
            gram3probs = probs["3gram"][index]
            gram2probs = probs["2gram"][index]
            dfaprobs = probs["dfa"][index]
            modelgram3diff = np.abs(modelprobs[:len(gram3probs)] - gram3probs)
            modeldfadiff = np.abs(modelprobs[:len(dfaprobs)] - dfaprobs)
            modelgram2diff = np.abs(modelprobs[:len(gram2probs)] - gram2probs)

            currentprobs = {
                exp_name: modelprobs,
                "3gram": gram3probs,
                "2gram": gram2probs,
                "DFA": dfaprobs,
                f"|{exp_name} - 3gram|": modelgram3diff,
                f"|{exp_name} - DFA|": modeldfadiff,
                f"|{exp_name} - 2gram|": modelgram2diff,
            }

            print(logitsmap(currentprobs, list(data[index]['input']), data[index]['vocab'], model=exp_name, id=f"{index}_probs"))
            print(logitsmap(currentprobs, list(data[index]['input']), data[index]['vocab'], model=exp_name, id=f"{index}_probs", offset=0))


    except Exception as e:
        print(e)
        print(f"skipping {exp_name}")


In [ ]:
accuracies = {}
for key1 in probs.keys():
    try:
        accuracies[key1] = get_greedy_dfa_accuracy(probs[key1], probs["dfa"])
    except:
        print(f"skipping {key1}")

In [ ]:
ce_losses = {}
for key1 in probs.keys():
    try:
        ce_losses[key1] = get_cross_entropy(probs[key1], probs["dfa"])
    except Exception as e:
        print(e)
        print(f"skipping {key1}")

In [ ]:
ce_losses

In [ ]:
# plot bar chart of ce_losses with labels

plt.figure(figsize=(32, 10))
plt.bar(ce_losses.keys(), ce_losses.values())
plt.title("KL (pmodel, pdfa)")

In [ ]:
# similar bar chart for accuracy
plt.figure(figsize=(32, 8))
plt.bar(accuracies.keys(), accuracies.values())
plt.title("Average Validity Accuracy")



In [ ]:
l1_losses = {}
for key1 in probs.keys():
    for key2 in probs.keys():
        if key1 == key2:
            continue
        try:
            l1_losses[(key1, key2)] = get_l1_loss(probs[key1], probs[key2], probs["dfa"], offset=0, max_len=100)
        except Exception as e:
            print(f"skipping {key1}, {key2}")
            print(e)

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def heatmap(l1_losses, title=""):
    # l1_losses is a dict of (key1, key2) -> l1_loss
    # we want to create a df where key1s are the index
    # and key2s are the columns
    key1s = sorted(list({key1 for key1, key2 in l1_losses.keys()}))
    key2s = sorted(list({key2 for key1, key2 in l1_losses.keys()}))
    df = pd.DataFrame(index=key1s, columns=key2s)
    for key1, key2 in l1_losses.keys():
        df.loc[key1, key2] = l1_losses[(key1, key2)]
    # replace nans with 0
    df = df.fillna(0)
    # heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(df, annot=True)
    # add title
    plt.title(f"L1 Loss (pmodel1, pmodel2) {title}")

In [ ]:
heatmap(l1_losses);

In [ ]:
heatmap(l1_losses, title="offset=200, max_len=300")

In [ ]:
heatmap(l1_losses, title="offset=0, max_len=100")

## Clustering plots of each model

In [ ]:
from sklearn.manifold import TSNE
# import PCA
from sklearn.decomposition import PCA
import numpy as np
from make_video import prepare_video
import seaborn as sns

In [ ]:
fill_types = ('o', '8', '*', 'v', 'X', '^', '<', '>','s', 'p',  'h', 'H', 'D', 'd', 'P')

In [ ]:
import matplotlib.cm as cm
colors = cm.viridis(np.linspace(0, 1, 12))

In [ ]:
colors = cm.Set1.colors +  ((0.0, 0.0, 0.0), cm.Set2.colors[0], cm.tab20b.colors[-1], cm.tab20b.colors[-2], cm.tab20b.colors[-3])

In [ ]:
factors = {
    9: (3, 3),
    13: (4, 4),
    5: (3, 3),
    3: (2, 2),
    1: (1, 1),
}

In [ ]:
def plot_clusters(folder, out_folder, reduction="tsne", layer_wise=False):
    results = get_results(os.path.join(folder, "generations", "200_test.txt"))
    print(f"plotting {folder}"  + " " + str(len(results)))
    os.makedirs(os.path.join(folder, "plots"), exist_ok=True)
    for i in range(2):
        example = results[i]
        t_hidden = len(example["hidden_outputs"][0])
        t_states = len(example["states"])
        n_layers = len(example["hidden_outputs"])
        # print(f"t_hidden: {t_hidden}, t_states: {t_states}, n_layers: {n_layers}")

        reducer = TSNE if reduction == "tsne" else PCA


        if layer_wise:
            Xs = [reducer(n_components=2).fit_transform(hidden_outputs[:t_states, :]) for hidden_outputs in example["hidden_outputs"]]
            X = np.concatenate(Xs, axis=0)
            X = X.reshape(n_layers, t_states, 2)
        else:
            hidden_outputs = np.concatenate(example["hidden_outputs"], axis=0)
            hidden_outputs = hidden_outputs[:, :t_states, :]
            X = reducer(n_components=2).fit_transform(hidden_outputs)
            X = X.reshape(n_layers, t_states, 2)

        X = X[:, :t_states, :]


        for t in range(20, 500, 10):
            a, _ = factors[n_layers]
            axes, fig = plt.subplots(a, a, figsize=(16, 16))
            # assert n_layers == 13
            axes.suptitle(f"T={t}, #states: "
                    + str(len(example["dfa"].dfa._transition_function))
                    + ", len(vocab): "
                    + str(len(example["dfa"].dfa.alphabet)))

            for layer in range(n_layers-1, -1, -1):
                X_layer = X[layer, :t]
                labels = np.array(example["states"][:t])
                chars = np.array(list(example["input"][:t]))
                vocab = example["vocab"][1:]

                ax = fig[layer // a, layer % a]
                for label in np.unique(labels):
                    if label == -1 or label == "-1":
                        continue
                    for ci, v in enumerate(example["dfa"].dfa.alphabet):
                        indices = np.where((labels == label) & (chars == v))
                        if len(indices) > 0:
                            ax.scatter(X_layer[indices, 0], X_layer[indices, 1], marker=fill_types[ci], c=colors[label])
                            # show legend
                # set title
                ax.set_title(
                    f"layer={layer}"
                )
            plot_path = os.path.join(folder, "plots/", f"e_{i}_{reduction}_out_label_t_{t}.png")
            plt.savefig(plot_path)
            plt.close()

        video_out_folder = os.path.join("video_outputs", out_folder)
        video_glob = os.path.join(folder, "plots/") + "/" + f"e_{i}_{reduction}_out_label_t_*.png"
        print(video_glob)
        print("preparing video")
        videopath = prepare_video(video_glob, video_out_folder, f"{i}_lw_{layer_wise}")
        print(f"done at {videopath}")


In [ ]:
data1 = get_results(exp_folders["transformer/8"] + "/generations/200_test.txt")[:20]

In [ ]:
from sklearn.cluster import KMeans

def get_cluster_labels(example, layer=4):
    t_hidden = len(example["hidden_outputs"][0])
    t_states = len(example["states"])
    n_layers = len(example["hidden_outputs"])
    # print(f"t_hidden: {t_hidden}, t_states: {t_states}, n_layers: {n_layers}")
    reducer = TSNE
    Xs = [reducer(n_components=2).fit_transform(hidden_outputs[:t_states, :]) for hidden_outputs in example["hidden_outputs"][layer:layer+1]]
    X = np.concatenate(Xs, axis=0)
    X = X.reshape(1, t_states, 2)
    X = X[:, :t_states, :]
    X = X[0]
    print(X.shape)
    # find seperating hyperplane
    kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
    labels = kmeans.labels_
    inputs = example["input"][:t_states]
    states = example["states"][:t_states]
    return list(zip(labels, inputs, states))

In [ ]:
label_info = get_cluster_labels(data1[1], layer=4)

for label, input, state in label_info:
    print(f"label={label} input={input} outstate={state}")

In [ ]:
path = plot_clusters(exp_folders["transformer/8"], "transformer/8", reduction="tsne", layer_wise=True)

In [ ]:
paths = []
for exp_name, folder in exp_folders.items():
    print(exp_name)
    try:
        path = plot_clusters(folder, exp_name, reduction="tsne", layer_wise=True)
        paths.append(path)
    except:
        print(f"skipping {exp_name}")

In [ ]:
results = get_results(os.path.join(exp_folders["transformer/8"], "generations", "200_test.txt"))

In [ ]:
results[0].keys()

In [ ]:
results[0]["dfa"].dfa._transition_function

In [ ]:
from probe import get_dfa_states
def plot_attention_map(example, folder, output_folder, id=0):
    t_states = len(example["states"])
    t_hidden = len(example["hidden_outputs"][0])
    t_states = len(example["states"])
    n_layers = len(example["hidden_outputs"])
    in_states = get_dfa_states(example["input"], example["dfa"], in_states=True)
    out_states = get_dfa_states(example["input"], example["dfa"], in_states=False)

    start = np.random.randint(0, t_states)
    end = min(start + 20, t_states)

    start = 0
    end = 80


    chars = list(example["input"])[start: end]
    print(chars)
    in_states = in_states[start: end]
    out_states = out_states[start: end]
    chars = list(zip(in_states, chars, out_states))

    os.makedirs(os.path.join(folder, "attention_plots"), exist_ok=True)

    for layer in range(8):
        print(layer)
        attentions = example["attention_scores"][layer][:, start:end, start:end] # 2 x T X T
        print(attentions.shape)
        fig, ax = plt.subplots(2, 1, figsize=(25, 25))
        # plot heatmap of attentions per head
        # use sns plot
        for head in range(attentions.shape[0]):
            sns.heatmap(attentions[head],
                        ax=ax[head],
                        xticklabels=chars,
                        yticklabels=chars,
                        annot=False)
            ax[head].set_title(f"layer={layer}, head={head}")
            ax[head].xaxis.tick_top()
            # make xticks rotated 90 degrees
            plt.setp(ax[head].get_xticklabels(), rotation=90)
            # save figure
        os.makedirs(os.path.join("attention_plots", output_folder, f"{id}"), exist_ok=True)
        path = os.path.join("attention_plots", output_folder, f"{id}/layer_{layer}_head_{head}.jpeg")
        plt.savefig(path)
            # ax[head].set_xticks(range(len(chars)))
            # ax[head].set_xticklabels(chars)
            # ax[head].set_yticks(range(len(chars)))
            # ax[head].set_yticklabels(chars)

In [ ]:
plot_attention_map(results[0], exp_folders["transformer/8"])

In [ ]:
paths = []
for exp_name, folder in exp_folders.items():
    print(exp_name)
    try:
        results = get_results(os.path.join(folder, "generations", "200_test.txt"))
        # plot_attention_map(results[0], folder, output_folder=exp_name, id=0)
        plot_attention_map(results[1], folder, output_folder=exp_name, id=1)
    except Exception as e:
        print(e)
        print(f"skipping {exp_name}")

In [ ]:
exp_folders["transformer/8"]

In [ ]:
results[0]["attention"]

In [ ]:
from probe import get_dfa_states
from bertviz import head_view, model_view
import torch

def visualize_attention_map(example, folder, id=0, output_folder="transformer/8"):
    t_states = len(example["states"])
    t_hidden = len(example["hidden_outputs"][0])
    t_states = len(example["states"])
    n_layers = len(example["hidden_outputs"])
    in_states = get_dfa_states(example["input"], example["dfa"], in_states=True)
    out_states = get_dfa_states(example["input"], example["dfa"], in_states=False)

    start = np.random.randint(0, t_states)
    end = min(start + 20, t_states)

    start = 0
    end = 80


    chars = list(example["input"])[start: end]
    print(chars)
    in_states = in_states[start: end]
    out_states = out_states[start: end]
    chars = list(zip(in_states, chars, out_states))
    chars = [str(text) for text in chars]
    attentions = [torch.tensor(scores[None, :, start:end, start:end]) for scores in example["attention_scores"]]
    html_head_view = head_view(attentions, chars, html_action='return')

    output_folder = os.path.join("attention_plots", output_folder)
    os.makedirs(output_folder, exist_ok=True)
    with open(os.path.join(output_folder, f"{id}.html"), 'w') as file:
        file.write(html_head_view.data)


In [ ]:
paths = []
for exp_name, folder in exp_folders.items():
    print(exp_name)
    try:
        results = get_results(os.path.join(folder, "generations", "200_test.txt"))
        visualize_attention_map(results[0], folder, id=0, output_folder=exp_name)
        visualize_attention_map(results[1], folder, id=1, output_folder=exp_name)
    except:
        print(f"skipping {exp_name}")

In [ ]:
visualize_attention_map(results[0], exp_folders["transformer/8"])

In [ ]:
html_head_view = head_view(attention, tokens, html_action='return')

with open("PATH_TO_YOUR_FILE/head_view.html", 'w') as file:
    file.write(html_head_view.data)

In [ ]:
len(results[0]["attention_scores"])